### Import libraries

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt
import seaborn

### Granger Kausalität für alle Staaten

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

# Read the datasets into DataFrames
military_df = pd.read_csv("data/military_expenditure.csv", skiprows=4)
gdp_df = pd.read_csv("data/GDP.csv", skiprows=4)
imports_df = pd.read_csv("data/imports.csv", skiprows=4)
exports_df = pd.read_csv("data/exports.csv", skiprows=4)

# Define the years to consider
years = [str(year) for year in range(2000, 2024)]

# Filter only relevant columns
military_data = military_df[['Country Name'] + years]
imports_data = imports_df[['Country Name'] + years]
gdp_data = gdp_df[['Country Name'] + years]
exports_data = exports_df[['Country Name'] + years]

# Differenzierung
countries = sorted(set(military_data['Country Name']) & set(gdp_data['Country Name']) & set(imports_data['Country Name']) & set(exports_data['Country Name']))

def differ(df):
    df_out = df.copy()
    for country in countries:
        row = df[df['Country Name'] == country].copy()
        row = pd.melt(
            row,
            id_vars=['Country Name'],
            var_name='Year',
            value_name='Money',
            value_vars=[str(year) for year in years]
        )
        row["Money"] = row["Money"].diff()
        row['Country Name'] = country  # Restore in case it was removed
        row = row.pivot(index="Country Name", columns="Year", values="Money")
        row.columns.name = None
        row = row.reset_index()

        for col in row.columns:
            if col in df_out.columns:
                df_out.loc[df_out["Country Name"] == country, col] = row.iloc[0][col]
    return df_out

military_data = differ(military_data)
imports_data = differ(imports_data)
exports_data = differ(exports_data)
gdp_data = differ(gdp_data)

# Define the years to consider
years = [str(year) for year in range(2001, 2024)]

# Filter only relevant columns
military_data = military_df[['Country Name'] + years]
imports_data = imports_df[['Country Name'] + years]
gdp_data = gdp_df[['Country Name'] + years]
exports_data = exports_df[['Country Name'] + years]

# Merge all datasets on 'Country Name'
merged_df = military_data.merge(imports_data, on='Country Name', suffixes=('_military', '_imports'))
merged_df = merged_df.merge(gdp_data, on='Country Name')
merged_df.rename(columns={year: f"{year}_gdp" for year in years}, inplace=True)
merged_df = merged_df.merge(exports_df, on='Country Name')
merged_df.rename(columns={year: f"{year}_exports" for year in years}, inplace=True)

# Compute Granger causality for each country
granger_results = []

for index, row in merged_df.iterrows():
    country = row['Country Name']
    military = row[[f'{year}_military' for year in years]].astype(float)
    imports = row[[f'{year}_imports' for year in years]].astype(float)
    gdp = row[[f'{year}_gdp' for year in years]].astype(float)

    if military.isnull().any() or imports.isnull().any() or gdp.isnull().any():
        print("Skip " + country + " because of missing data")
        continue  # Skip countries with missing data

    try:
        data = pd.DataFrame({"imports": imports.values, "military": military.values})
        # Granger test: does military spending help predict imports?
        test_result = grangercausalitytests(data, maxlag=3)
        for lag in range(1, 4):
            p_value = round(test_result[lag][0]['ssr_ftest'][1], 4)
            granger_results.append({
                'Country': country,
                'KPI': 'Import',
                'Lag': lag,
                'P-Value': p_value
            })
    except Exception:
        pass
        # continue  # Skip if test fails

    try:
        data = pd.DataFrame({"exports": exports.values, "military": military.values})
        # Granger test: does military spending help predict exports?
        test_result = grangercausalitytests(data, maxlag=3)
        for lag in range(1, 4):
            p_value = round(test_result[lag][0]['ssr_ftest'][1], 4)
            granger_results.append({
                'Country': country,
                'KPI': 'Export',
                'Lag': lag,
                'P-Value': p_value
            })
    except Exception:
        pass
        # continue  # Skip if test fails

    try:
        data = pd.DataFrame({"gdp": gdp.values, "military": military.values})
        # Granger test: does military spending help predict gdp?
        test_result = grangercausalitytests(data, maxlag=3)
        for lag in range(1, 4):
            p_value = round(test_result[lag][0]['ssr_ftest'][1], 4)
            granger_results.append({
                'Country': country,
                'KPI': 'GDP',
                'Lag': lag,
                'P-Value': p_value
            })
    except Exception:
        pass
        # continue  # Skip if test fails

# Convert results to DataFrame
granger_df = pd.DataFrame(granger_results)

# Display top results
print(granger_df.sort_values(by='P-Value').head(20))

In [ ]:
import pandas as pd
import pycountry
import plotly.express as px

df_map = granger_df[granger_df["KPI"] == "Import"]
df_map = df_map.loc[df_map.groupby('Country')['P-Value'].idxmin()]
print(df_map)

# 4) Interaktive Choropleth-Karte
fig = px.choropleth(
    df_map,
    locations='Country',
    locationmode='country names',
    color='P-Value',
    hover_name='Country',
    hover_data=['Lag'],
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Granger Kausalität: Militärausgaben -> Import% des GDP'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

### Granger Kausalität: Militärausgaben -> Export% Anteil an GDP

In [ ]:
import pandas as pd
import pycountry
import plotly.express as px

df_map = granger_df[granger_df["KPI"] == "GDP"]
df_map = df_map.loc[df_map.groupby('Country')['P-Value'].idxmin()]
print(df_map)

# 4) Interaktive Choropleth-Karte
fig = px.choropleth(
    df_map,
    locations='Country',
    locationmode='country names',
    color='P-Value',
    hover_name='Country',
    hover_data=['Lag'],
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Granger Kausalität: Militärausgaben -> Export% des GDP'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()